In [33]:
import numpy as np
from datasets import load_dataset
dataset = load_dataset("sst", "default")
train = dataset['train'].to_pandas().to_numpy()
test=dataset['test'].to_pandas().to_numpy()
train

array([["The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .",
        0.6944400072097778,
        "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.",
        '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'],
       ["The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .",
        0.833329975605011,
        "The|gorgeously|elaborate|continuation|of|``|The|Lord|of|the|Rings|''|trilogy|is|so|hug

In [34]:
sentences=train[:,0]
labels=train[:,1]
bag_of_words={}
for sentence in sentences:
    words = sentence.split()
    for word1,word2 in zip(words,words[1:]):
        bigram=(word1,word2)
        bag_of_words[bigram]=bag_of_words.get(bigram,0)+1
bag_of_words=dict(sorted(bag_of_words.items(),key=lambda kv:-kv[1] ))
bag_of_words

{('of', 'the'): 821,
 (',', 'but'): 702,
 (',', 'and'): 666,
 ('in', 'the'): 414,
 ('It', "'s"): 398,
 ('it', "'s"): 388,
 ('is', 'a'): 371,
 ('the', 'film'): 340,
 ('of', 'a'): 332,
 (',', 'the'): 320,
 ('and', 'the'): 269,
 ('to', 'the'): 260,
 (',', 'it'): 248,
 ('to', 'be'): 243,
 ('the', 'movie'): 198,
 ("'s", 'a'): 197,
 ('with', 'a'): 191,
 ('for', 'the'): 190,
 ('on', 'the'): 185,
 ('but', 'it'): 185,
 ('in', 'a'): 180,
 ('as', 'a'): 178,
 ('and', 'a'): 173,
 ('it', 'is'): 169,
 ('does', "n't"): 165,
 ('The', 'film'): 146,
 ('a', 'movie'): 139,
 ('that', 'it'): 134,
 ('one', 'of'): 133,
 ('with', 'the'): 132,
 ('film', 'is'): 129,
 ('is', "n't"): 128,
 (',', 'a'): 127,
 ('that', "'s"): 125,
 ('it', '.'): 124,
 ('film', '.'): 121,
 ('of', 'its'): 118,
 ('for', 'a'): 117,
 ('like', 'a'): 115,
 ('movie', 'is'): 109,
 ('at', 'the'): 108,
 ('movie', '.'): 107,
 ('This', 'is'): 106,
 ('film', 'that'): 106,
 (',', 'this'): 105,
 ('from', 'the'): 105,
 ('as', 'the'): 105,
 ('the', 'mos

In [35]:
x_train_sentence_vectors = np.zeros((len(sentences), len(bag_of_words)), dtype=int)
bi_gram_index = {bi_gram: index for index, bi_gram in enumerate(bag_of_words)}
for i, sentence in enumerate(sentences):
    words = sentence.split()
    bigrams=set()
    for word1,word2 in zip(words,words[1:]):
        bigrams.add((word1,word2))
    for bi_gram in bigrams:
        if bi_gram in bi_gram_index:
            index = bi_gram_index[bi_gram]
            x_train_sentence_vectors[i, index] = 1
print(x_train_sentence_vectors.shape)


(8544, 87249)


In [36]:
del x_train_sentence_vectors
del bi_gram_index
del bag_of_words

In [37]:
print(labels)
def maping(a):
    if a<=0.2:
        return 0       #'very negative'
    if a<=0.4 and a>0.2:
        return 1       # "negative"
    if a<=0.6 and a>0.4:
        return 2       # "neutral"
    if a<=0.8 and a>0.6:
        return 3       # "positive"
    if a<=1 and a>0.8:
        return 4       # "very positive"
y_train= np.vectorize(maping)(labels)

[0.6944400072097778 0.833329975605011 0.625 ... 0.75 0.13888999819755554
 0.3472200036048889]


In [40]:
class logistic_regretion:
    def __init__(self):
        self.bias = None
        self.weights = None
        self._bag_of_words=None
        self._sentence_vector=None
        self.accuracy=None
        self._labels=None
    def one_hot_encode(self, y, num_classes):
        one_hot_y = np.zeros((len(y), num_classes))
        one_hot_y[np.arange(len(y)), y] = 1
        return one_hot_y
    def softmax(self,z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    def _x_vectorized(self,sentences):
        x_train_sentence_vectors = np.zeros((len(sentences), len(self._bag_of_words)), dtype=int)
        for i, sentence in enumerate(sentences):
            words = sentence.split()
            bigrams=set()
            for word1,word2 in zip(words,words[1:]):
                 bigrams.add((word1,word2))
            for bi_gram in bigrams:
                if bi_gram in self._bag_of_words:
                    index = self._bag_of_words[bi_gram]
                    x_train_sentence_vectors[i, index] = 1
        return x_train_sentence_vectors
        
    def fit(self,sentences,labels):
        bag_of_words={}
        for sentence in sentences:
            words = sentence.split()
            for word1,word2 in zip(words,words[1:]):
                bigram=(word1,word2)
                bag_of_words[bigram]=bag_of_words.get(bigram,0)+1
        bag_of_words=dict(sorted(bag_of_words.items(),key=lambda kv:-kv[1] ))
        self._bag_of_words= {bi_gram: index for index, bi_gram in enumerate(bag_of_words)}
        self._sentence_vector=self._x_vectorized(sentences)
        self._labels=labels
    def compile(self,n_epochs,n_batches,learning_rate=0.001,l2_lambda=0.01):
        print("starting training model...This could take a while")
        num_classes = len(np.unique(self._labels))
        self.weights = np.zeros((self._sentence_vector.shape[1],num_classes))
        y_one_hot = self.one_hot_encode(self._labels, num_classes)
        self.bias = np.zeros((1, num_classes))
        for epoch in range(n_epochs):
            indices = np.random.permutation(self._sentence_vector.shape[0])
            X_shuffled = self._sentence_vector[indices]
            y_shuffled = y_one_hot[indices]
            for i in range(0, self._sentence_vector.shape[0], n_batches):
                X_batch = X_shuffled[i:i + n_batches]
                y_batch = y_shuffled[i:i + n_batches]
                for j in range(len(X_batch)):
                    x_sample = X_batch[j].reshape(1, -1)
                    y_sample = y_batch[j].reshape(1, -1)
                    z=np.dot(x_sample,self.weights)+self.bias
                    y_pred = self.softmax(z)    
                    dz = y_pred - y_sample
                    dw = np.dot(x_sample.T, dz) + 2 * l2_lambda * self.weights 
                    db = dz
                    self.weights -= learning_rate * dw
                    self.bias -= learning_rate * db.flatten()
                print(f"Epoch {epoch + 1}/{n_epochs},Batch {i // n_batches + 1}/{self._sentence_vector.shape[0] // n_batches},Accuracy : {self._accuracy(X_batch,np.argmax(y_batch, axis=1))}")
        self.accuracy=self._accuracy(self._sentence_vector,np.argmax(y_one_hot, axis=1))
    def _predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)
    def _accuracy(self, x, y):
        predictions = self._predict(x)
        # print(predictions.shape," dah shape el pred : ",y.shape)
        correct_predictions = np.sum(predictions == y)
        accuracy = correct_predictions / len(y)
        return accuracy
    def predict(self,sentences):
        x_vector=self._x_vectorized(sentences)
        return self._predict(x_vector)

In [42]:
model=logistic_regretion()
model.fit(sentences,y_train)
model.compile(n_epochs=50,n_batches=sentences.shape[0]//8)
print(model.accuracy)

starting training model...This could take a while
Epoch 1/50,Batch 1/8,Accuracy : 0.41198501872659177
Epoch 1/50,Batch 2/8,Accuracy : 0.4250936329588015
Epoch 1/50,Batch 3/8,Accuracy : 0.33239700374531833
Epoch 1/50,Batch 4/8,Accuracy : 0.4737827715355805
Epoch 1/50,Batch 5/8,Accuracy : 0.39325842696629215
Epoch 1/50,Batch 6/8,Accuracy : 0.46254681647940077
Epoch 1/50,Batch 7/8,Accuracy : 0.35767790262172283
Epoch 1/50,Batch 8/8,Accuracy : 0.3061797752808989
Epoch 2/50,Batch 1/8,Accuracy : 0.37640449438202245
Epoch 2/50,Batch 2/8,Accuracy : 0.3895131086142322
Epoch 2/50,Batch 3/8,Accuracy : 0.3642322097378277
Epoch 2/50,Batch 4/8,Accuracy : 0.3951310861423221
Epoch 2/50,Batch 5/8,Accuracy : 0.38670411985018727
Epoch 2/50,Batch 6/8,Accuracy : 0.3295880149812734
Epoch 2/50,Batch 7/8,Accuracy : 0.3352059925093633
Epoch 2/50,Batch 8/8,Accuracy : 0.35580524344569286
Epoch 3/50,Batch 1/8,Accuracy : 0.3398876404494382
Epoch 3/50,Batch 2/8,Accuracy : 0.3529962546816479
Epoch 3/50,Batch 3/8,Acc

In [43]:
sentences=test[:,0]
labels=test[:,1]
y_test_true=np.vectorize(maping)(labels)
y_test_prediction=model.predict(sentences)
acc=y_test_prediction==y_test_true
x,c=np.unique(acc,return_counts=True)
print(x," ",c)
print("Accuracy: ",(c[1]/y_test_prediction.shape)*100)

[False  True]   [1485  725]
Accuracy:  [32.80542986]


In [51]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
vectorizer = TfidfVectorizer() 
X_train_vectorized = vectorizer.fit_transform(train[:,0])
X_test_vectorized = vectorizer.transform(test[:,0])

sgd_classifier = SGDClassifier( max_iter=50, alpha=0.001,random_state=42)  # Using logistic regression loss for SGD


sgd_classifier.fit(X_train_vectorized, y_train)


sgd_pred = sgd_classifier.predict(X_test_vectorized)

print("SGD Classifier Accuracy:", accuracy_score(y_test_true, sgd_pred))


SGD Classifier Accuracy: 0.4108597285067873


In [52]:
logistic_regression = LogisticRegression(max_iter=50, random_state=42)
logistic_regression.fit(X_train_vectorized, y_train)
logistic_pred = logistic_regression.predict(X_test_vectorized)
print("Logistic Regression Accuracy:", accuracy_score(y_test_true, logistic_pred))

Logistic Regression Accuracy: 0.4099547511312217


C:\Users\zeyad\Desktop\computer communication\compu 3rd year\second term\natural language processing\assignment1\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
